In [35]:
from fpdf import FPDF
from deep_translator import GoogleTranslator
import pdfplumber

# Configuration
INPUT_PDF = "test1.pdf"
OUTPUT_PDF = "testtable_translated_output.pdf"
SOURCE_LANG = "english"
TARGET_LANG = "hindi"
FONT_PATH = "NotoSans-Regular.ttf"  # Path to TTF font


In [40]:
# TRANSLATION
def translate_table(table, source, target):
    translator = GoogleTranslator(source=source, target=target)
    translated = []
    for row in table:
        translated_row = [translator.translate(cell) if cell else "" for cell in row]
        translated.append(translated_row)
    return translated

# PDF GENERATOR
class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.add_font("NotoSans", "", FONT_PATH, uni=True)
        self.set_font("NotoSans", size=10)

    def get_max_line_count(self, row, col_widths):
        max_lines = 1
        for i, cell in enumerate(row):
            text = cell or ""
            string_width = self.get_string_width(text)
            max_lines = max(max_lines, int(string_width / col_widths[i]) + 1)
        return max_lines

    def render_table(self, table, margin=10, padding=3):
        self.add_page()
        self.set_left_margin(margin)
        self.set_right_margin(margin)
        table_width = self.w - 2 * margin

        num_cols = len(table[0])
        col_widths = [table_width / num_cols for _ in range(num_cols)]
        line_height = self.font_size + padding

        for row in table:
            x_start = self.get_x()
            y_start = self.get_y()

            max_lines = self.get_max_line_count(row, col_widths)
            row_height = line_height * max_lines

            # Draw each cell
            for i, cell in enumerate(row):
                text = cell or ""
                x = self.get_x()
                y = self.get_y()

                self.multi_cell(
                    col_widths[i],
                    line_height,
                    text,
                    border=1,
                    align="L"
                )

                # Move to right of current cell
                self.set_xy(x + col_widths[i], y)

            self.set_xy(x_start, y_start + row_height)

# MAIN
def main():
    all_translated_tables = []

    with pdfplumber.open(INPUT_PDF) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                if table:
                    translated = translate_table(table, SOURCE_LANG, TARGET_LANG)
                    all_translated_tables.append(translated)

    pdf = PDF()
    for table in all_translated_tables:
        pdf.render_table(table)

    pdf.output(OUTPUT_PDF)
    print("✅ Clean, translated table PDF saved:", OUTPUT_PDF)

# RUN
if __name__ == "__main__":
    main()

✅ Clean, translated table PDF saved: testtable_translated_output.pdf
